In [1]:
import numpy as np
import igraph as ig

In [2]:
import os
import numpy as np
import pandas as pd
import scipy.io as sio

In [3]:
def loadcache(name):
    out = np.load(f"{name}.npy", allow_pickle=True)
    if isinstance(out, np.ndarray):
        return out
    else:
        return out.item()

# Load Shot Information

start, time,

In [4]:
shot_boundaries = pd.read_csv("hp_shot_boundaries.videvents", sep=' ')

In [5]:
shot_boundaries

,start,time,CVHCI_VIDEOEVENTS_V1
0,115,4.796,CUT
1,910,37.955,CUT
2,1147,47.839,CUT
3,1290,53.804,CUT
4,1384,57.724,CUT
...,...,...,...
2519,206711,8621.571,CUT
2520,206804,8625.450,CUT
2521,206907,8629.746,CUT
2522,219038,9135.710,CUT


In [6]:
def getShots(sceneStart, sceneEnd):
    res = []
    
    for shot in range(len(shot_boundaries)):
        shotEnd = shot_boundaries.iloc[shot]['time']
                        
        if shot != 0:
            shotStart = shot_boundaries.iloc[shot-1]['time']
        else:
            shotStart = 0

        # If the shot start after the scene starts
        if shotStart >= sceneStart and shotEnd <= sceneEnd:
            res.append(shot)

        # If the scene starts AFTER shot start but continues after shot ENDS too (the main condition)
        elif shotStart <= sceneStart and shotEnd >= sceneStart:
            res.append(shot)
            
        elif shotStart <= sceneEnd and shotEnd >= sceneEnd:
            res.append(shot)
            
#             elif shot == 0 and start >= 0:
#                 res.append(shot)
        
    return res

# Load Scene Information

Get scene breaks, start shot, end shot, # shots

In [7]:
scene_breaks = sio.loadmat("scene_breaks.mat")['scene_breaks']
scene_breaks = scene_breaks - 1 # Convert from matlab indexing to python
scene_breaks = np.array(list(filter(lambda x: x[2], scene_breaks)))

In [8]:
scene_breaks

array([[   0,   25,   25],
       [  26,   55,   29],
       [  56,   83,   27],
       [  84,  120,   36],
       [ 121,  167,   46],
       [ 168,  195,   27],
       [ 196,  214,   18],
       [ 215,  234,   19],
       [ 235,  248,   13],
       [ 249,  281,   32],
       [ 282,  299,   17],
       [ 300,  339,   39],
       [ 340,  354,   14],
       [ 355,  361,    6],
       [ 362,  379,   17],
       [ 380,  386,    6],
       [ 387,  403,   16],
       [ 404,  428,   24],
       [ 429,  451,   22],
       [ 452,  481,   29],
       [ 482,  499,   17],
       [ 500,  527,   27],
       [ 528,  572,   44],
       [ 573,  592,   19],
       [ 593,  650,   57],
       [ 651,  663,   12],
       [ 664,  684,   20],
       [ 685,  704,   19],
       [ 705,  737,   32],
       [ 738,  799,   61],
       [ 800,  809,    9],
       [ 810,  860,   50],
       [ 861,  877,   16],
       [ 878,  954,   76],
       [ 955,  963,    8],
       [ 964,  987,   23],
       [ 988, 1013,   25],
 

Get start time, end time of those shots

In [9]:
start_times = []
end_times = []

In [10]:
for scene_break in scene_breaks:
    start_shot, end_shot, nshots = scene_break
    
    start_time, end_time = shot_boundaries.iloc[start_shot]['time'], shot_boundaries.iloc[end_shot-1]['time']
    
    start_times.append(start_time)
    end_times.append(end_time)

In [11]:
scene_break_times = np.hstack([scene_breaks, np.array(start_times).reshape(-1, 1), np.array(end_times).reshape(-1, 1)])

In [12]:
def getShotsScene(sceneNum):
    shotStart, shotEnd, numShots = scene_breaks[sceneNum]
    out = [i for i in range(shotStart, shotEnd)]
    
    return out

# Load Ground Truth - Shots Per Chapter

In [13]:
import json

In [14]:
alignment = None

with open("hp_b2m_gtalign.json", "r") as f:
    alignment = json.load(f)['b2m_hp1']

In [15]:
def toTime(time):
    min, s = time.split(":")
    min = int(min)
    s = int(s)
    
#     print(h, min)
    
    return 60 * min + s

Get shots per chapter

In [16]:
shotsPerChapterGT = []

In [17]:
for chap in alignment:
    start = chap['t_start']
    end = chap['t_end']
    
    shotChapter = getShots(start, end)
    
    shotsPerChapterGT.append(shotChapter)
    
#     print(start, end, shotChapter)
#     print("-----")

In [18]:
for idk in shotsPerChapterGT:
    print(idk)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
[46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116]
[116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209]
[215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 2

Get chapter per Shot (same thing, but as a dict)

In [31]:
from collections import defaultdict

In [32]:
shot_chapter = defaultdict(lambda: -1)

for i, chapter in enumerate(shotsPerChapterGT):
    for shot in chapter:
        shot_chapter[shot] = i+1

# Calculate Scene Detection Confidence

1 1 1 1 1 2 2 | 2 2 2 2 2| 2 2 | 3 3 3 3 3

In [33]:
from collections import Counter

In [37]:
correctShots = 0
tot = 0

for i, scene in enumerate(scene_breaks):
    
    start_shot, end_shot, _ = scene
    scenes_shots = np.arange(start_shot+1, end_shot)
    
    scene_chapters = dict(Counter(list(map(lambda x: shot_chapter[x], scenes_shots))))
    print(list(map(lambda x: shot_chapter[x], scenes_shots)))
    
    scene_chapter = sorted(scene_chapters.keys(), key=lambda x: scene_chapters[x])[-1]
    
    for shot in scenes_shots:
        if shot_chapter[shot] == scene_chapter:
            correctShots += 1
            
    tot += len(scenes_shots)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, -1, -1, -1, -1]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

In [36]:
correctShots / tot

0.9785223367697594